# Merge and Groupby
Duncan Callaway

This notebook gives an introduction to using Pandas' `merge` and `groupby` methods.  

In [1]:
import pandas as pd
import numpy as np

## Row and column labels
The columns are identified with a list of values.  Let's look at the fruit data set again:

In [2]:
fruit_info_df = pd.read_csv('fruit_info.csv', index_col= False)
fruit_info_df

,fruit,color,weight
0,apple,red,120
1,banana,yellow,150
2,orange,orange,250
3,raspberry,pink,15


### Q: How do I print out just the columns?

In [3]:
fruit_info_df.columns

Index(['fruit', 'color', 'weight'], dtype='object')

### Q: And the rows?

The rows are similarly labeled:

In [4]:
fruit_info_df.index

RangeIndex(start=0, stop=4, step=1)

## Merging
Lets make another data frame and tack it on to the first

In [5]:
price_df = pd.DataFrame({'price':[0.5, 0.65, 1, 0.15],
                        'frut':['apple', 'banana', 'orange', 'rasberry']})
price_df

,price,frut
0,0.50,apple
1,0.65,banana
2,1.00,orange
3,0.15,rasberry


Now let's blindly merge:

In [6]:
pd.merge(price_df,fruit_info_df)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

What went wrong?

First, we didn't spell fruit correctly.  Two ways to fix.  First, specify the columns directly:

In [7]:
pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit')

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250


Second, fix the spelling and *don't* tell pandas.  In this case pandas works to figure out what's in common.

In [8]:
price_df.columns[0]='fruit'

TypeError: Index does not support mutable operations

Bummer!  Can't mutate index values.  What to do?

In [9]:
col_list = list(price_df.columns)
col_list

['price', 'frut']

In [10]:
col_list[1] = 'fruit'

In [11]:
price_df.columns = col_list
price_df

,price,fruit
0,0.50,apple
1,0.65,banana
2,1.00,orange
3,0.15,rasberry


In [12]:
pd.merge(fruit_info_df,price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00


Note we can use different syntax:

In [13]:
fruit_info_df.merge(price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00


### Q: Now we're still missing raspberries -- why?

Again, spelling error in the new frame.  Let's fix:

In [14]:
price_df.loc[3,'fruit'] = 'raspberry'

Note we could change individual entries in the data frame itself.  They are mutable.

In [15]:
fruit_info_df.merge(price_df)

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00
3,raspberry,pink,15,0.15


Another few things to takeaway from this
1. Inner merge -- the default -- is brutal.  That is, it'll drop data without telling you.  Other ways to merge are a little less draconian.
2. It's important to review your results.  How many rows do you expect?  How many do you actually get?  Did something important get chucked out?  The ensuing solutions are the non-glamorous tasks of data cleaning.

Note, there are other commands -- `join`, `concat`, and these do similar things to `merge`.

I've found merge seems to work well for most purposes.

FWIW, `pd.concat` seems to be a little more brute force -- requires more careful syntax, but likely does unexpected things less often once you understand the syntax.

In [16]:
merged_df = fruit_info_df.merge(price_df)
merged_df

,fruit,color,weight,price
0,apple,red,120,0.50
1,banana,yellow,150,0.65
2,orange,orange,250,1.00
3,raspberry,pink,15,0.15


## Merge Types: Inner, Outer, Left, Right

Let's load up the dataframe again and experiment with different merge types:

In [17]:
fruit_info_df = pd.read_csv('fruit_info.csv', index_col= False)
price_df = pd.DataFrame({'price':[0.5, 0.65, 1, 0.15],
                        'frut':['apple', 'banana', 'orange', 'rasberry']})

In [18]:
merged_df_inner = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'inner')
merged_df_inner

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250


That's what we got above.  `pd.merge` gives an inner join by default.

### Q: Try doing an outer, left, and right merge. 

In [22]:
merged_df_outer = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'outer')
merged_df_outer

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120.0
1,0.65,banana,banana,yellow,150.0
2,1.00,orange,orange,orange,250.0
3,0.15,rasberry,NaN,NaN,NaN
4,NaN,NaN,raspberry,pink,15.0


You can see we kept *every* row from both dataframes, and populated with NaNs where keys don't match.

Let's try left and right:

In [23]:
merged_df_left = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'left')
merged_df_left

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120.0
1,0.65,banana,banana,yellow,150.0
2,1.00,orange,orange,orange,250.0
3,0.15,rasberry,NaN,NaN,NaN


In [24]:
merged_df_right = pd.merge(price_df,fruit_info_df, left_on = 'frut', right_on = 'fruit', how = 'right')
merged_df_right

,price,frut,fruit,color,weight
0,0.50,apple,apple,red,120
1,0.65,banana,banana,yellow,150
2,1.00,orange,orange,orange,250
3,NaN,NaN,raspberry,pink,15


We can streamline by replacing the index number with the fruit column.  

### Q: in the following, what's the `inplace` command for?  

In [25]:
merged_df.set_index('fruit', inplace = True)
merged_df

,color,weight,price
fruit,,,
apple,red,120,0.50
banana,yellow,150,0.65
orange,orange,250,1.00
raspberry,pink,15,0.15


### A: It means the re-defined dataframe is assigned to the original name.  
This is advantageous in memory constrained situations.  

## Multilevel indexing 

We can also assign "multilevel" column or row names, like so:

In [26]:
levels = [('categorical', 'color'),('quantitative', 'weight'),('quantitative','price')]
levels

[('categorical', 'color'),
 ('quantitative', 'weight'),
 ('quantitative', 'price')]

Note the  use of tuples (sets of values in parentheses) in setting up multiindex.  This will come again later.  

In [27]:
merged_df.columns = pd.MultiIndex.from_tuples(levels)
merged_df

categorical quantitative      
                color       weight price
fruit                                   
apple             red          120  0.50
banana         yellow          150  0.65
orange         orange          250  1.00
raspberry        pink           15  0.15

Now we have categories and subcategories of columns:

In [28]:
merged_df['quantitative']

,weight,price
fruit,,
apple,120,0.50
banana,150,0.65
orange,250,1.00
raspberry,15,0.15


### Q: How can we get data from an individual column?
Aim to get the `weight` column:

In [29]:
merged_df[('quantitative','weight')]

fruit
apple        120
banana       150
orange       250
raspberry     15
Name: (quantitative, weight), dtype: int64

## Advanced multilevel (did not do in lecture) 
Note, we can also drop and add things.  With multilevel indexing things get a little tricky.  

First, we can drop everything from the top level:

In [30]:
merged_test_df = merged_df.drop(columns=[('quantitative',)], axis = 1)
merged_test_df

,categorical
,color
fruit,
apple,red
banana,yellow
orange,orange
raspberry,pink


Note that I put the column identifier inside the parens, like a tuple, but it's not essential there.

However if we want to drop only a column from the second level, we get an error without the tuple syntax:

In [31]:
merged_test_df = merged_df.drop(columns=[('quantitative','price')], axis = 1)
merged_test_df

,categorical,quantitative
,color,weight
fruit,,
apple,red,120
banana,yellow,150
orange,orange,250
raspberry,pink,15


We can also drop rows: 

In [32]:
merged_df.drop(index=[('apple')], axis = 0, inplace = True)
merged_df

categorical quantitative      
                color       weight price
fruit                                   
banana         yellow          150  0.65
orange         orange          250  1.00
raspberry        pink           15  0.15

Note indexing multilevels with `.loc` gets a little tricky.  The thing to keep in mind is that you're working with tuples in each index location:

In [33]:
merged_df.loc['banana', ('quantitative', 'price')]

0.65

If you leave an entry of the tuple empty you get all values.  

In [34]:
merged_df.loc['banana', ('quantitative', )]

weight    150.00
price       0.65
Name: banana, dtype: float64

You can also loop through the columns of the multilevel data frame like this: 

In [35]:
for i, j in merged_df:
    print(merged_df.loc['banana', (i, j)])

yellow
150
0.65


Some added thoughts:
1. Multilevel indexing works for columns and index
2. It can be a powerful way to summarize your data and quickly reference subsets of it.
4. However it can also be a colossal pain in the rear -- indexing with multilevel is often very hard to parse and debug.

## Groupby

First, let's have another look at today's power point file.  Now we'll learn about how groupby works.

Back to the notebook, let's make a toy DF (example taken from Wes McKinney's [Python for Data Analysis](http://proquest.safaribooksonline.com.libproxy.berkeley.edu/book/programming/python/9781491957653):

In [36]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.373099,0.480167
1,a,two,0.497649,1.066077
2,b,one,-0.956532,2.327144
3,b,two,-0.614023,1.335843
4,a,one,-0.688224,0.682344


Let's group just the `data1` column by the `key1` column. A call to [`groupby`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) does that.  

Note, the syntax is to begin by invoking the portion of the dataframe we want to group (here, `df['data1']`), then we apply the groupby method with the portion of hte dataframe we want to group on (here `df['key1']`)

What is the object that results?

In [37]:
grouped = df['data1'].groupby(df['key1'])
grouped

As we see, it's not simply a new DataFrame. Instead, it's an object, in this case `SeriesGroupBy`.  We'll see in a moment that if we group many columns of data we get a `DataFrameGroupBy` object.

To look inside we need to use different syntax.  The specific thing we're looking for are the groups of the object...but let's tab in to the grouped object to see what's there.

In [38]:
grouped.groups

{'a': [0, 1, 4], 'b': [2, 3]}

That gave us the groups (a and b) and the indices of elements in the groups, but nothing else. 

You can see this structure looks like a dict.  a and b are the keys, and the data are lists associated with each key -- the values.  

But the `grouped` object is capable of making computations across all groups -- this is where it gets powerful.   

We can try things like `sum`, `min` and `max`.

Notice if you don't put the parens after the method, pandas returns information about what the method does, but not it's actual output.  

In [39]:
grouped.sum()

key1
a   -0.563673
b   -1.570555
Name: data1, dtype: float64

You can also pass `numpy` functions into the aggregate command.

But it can be informative to look at what's inside. We can iterate over a `groupby` object, as we iterate we get pairs of `(name, group)`, where the `group` is either a `Series` or a `DataFrame`, depending on whether the `groupby` object is a `SeriesGroupBy` (as above) or a `DataFrameGroupBy` (see below).

Something quirky to note about the interaction between the grouped object and the for loop structure below: we're going to define variables `name` and `group` as being things in `grouped`.  But there are no `name` or `group` attributes associated with the `grouped` object.  

In [40]:
for name, group in grouped:
    print('Name:', name)
    display(group)

Name: a


0   -0.373099
1    0.497649
4   -0.688224
Name: data1, dtype: float64

Name: b


2   -0.956532
3   -0.614023
Name: data1, dtype: float64

We can group on multiple keys, and the result is grouping by tuples:

In [41]:
g2 = df['data1'].groupby([df['key1'], df['key2']])
g2

In [42]:
g2.groups

{('a', 'one'): [0, 4], ('a', 'two'): [1], ('b', 'one'): [2], ('b', 'two'): [3]}

Now we have a groupby object that has tuples as the keys.  

In [43]:
g2.mean()

key1  key2
a     one    -0.530661
      two     0.497649
b     one    -0.956532
      two    -0.614023
Name: data1, dtype: float64

### Aside (did not do in lecture)
We can also group the entire dataframe -- not just one column of it -- on a single key. This results in a `DataFrameGroupBy` object as the result:

In [ ]:
k1g = df.groupby('key1')
k1g

In [ ]:
k1g.groups

That output actually looks a lot like the output when we were only grouping one of the columns of the dataframe.  But there is actually more information in the group itself.  

In [ ]:
for name, group in k1g:
    print('Name:', name)
    display(group)

The contents of each group is another dataframe.

In [ ]:
k1g.mean()

Where did column `key2` go in the mean above? It's a *nuisance column*, which gets automatically eliminated from an operation where it doesn't make sense (such as a numerical mean).

### Aside (did not do in lecture): Grouping over a different dimension

Above, we've been grouping data along the rows, using column keys as our selectors.  

But we can also group along the *columns*, 

What's even more cool?  We can group by *data type*.

Here we'll group along columns, by data type:

In [ ]:
df.dtypes

In [ ]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    display(group)

## Using groupby to re-ask our question 
*(did not do this in lecture, instead did CAISO forecasting error example)*
Which hour had the lowest average wind production?

In [ ]:
cds = pd.read_csv('CAISO_2017to2018_stack.csv', index_col= 0)

In [ ]:
cds.head()

It will help to have a column of hour of day values:

In [ ]:
cds_time = pd.to_datetime(cds.index)
type(cds_time)

Let's add that list of values into the data frame.

In [ ]:
cds['hour'] = cds_time.hour

In [ ]:
cds.head(10)

### Q: What groupby syntax would you use to arrange the data...
...so that you can examine production by hour and source?

See if you can do it yourself: we want to group MWh values by source AND hour.

In [ ]:
cds_grouped = cds['MWh'].groupby([cds['Source'],cds['hour']])

### Q: How to get *all* the means for all sources and hours?

Didn't need to do any fancy logical indexing or looping!

In [ ]:
cds_grouped.mean()

Now it would be nice to see that information in a dataframe, wouldn't it?

In [ ]:
averages = pd.DataFrame(cds_grouped.mean())

In [ ]:
averages

And lo and behold, we have a multilevel index for the rows!

In [ ]:
averages.loc[('WIND TOTAL',),:]

But now we can look at other sources

In [ ]:
averages.loc[('SMALL HYDRO',),:]

Let's plot:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(averages.loc[('SMALL HYDRO',),:]);

In [ ]:
plt.plot(averages.loc[('GEOTHERMAL',),:]);

In [ ]:
plt.plot(averages.loc[('SOLAR PV',),:]);